In [109]:
# gs://ds_tianlong/prepared_files_for_amir
import pandas as pd
import numpy as np
from google.cloud import bigquery

In [70]:
myData = pd.read_csv('2021_10_20_products_series_labelled.csv')

In [6]:
myData

,omsid,product_name,description,brand,bullets,series_name
0,313296145,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,High pressure ceramic cartridge|Solid brass co...,Jax
1,313296138,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,"Solid brass construction|Supply lines, valves ...",Jax
2,313296149,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,Standard plumbing installation|ADA compliant|c...,Jax
3,313296158,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,ADA compliant|Solid brass construction|High pr...,Jax
4,315696655,AN-4500 and AN-4512 Aria Single Handle Single ...,NaN,NaN,Single-hole installation|cUPC certified|All mo...,Aria
...,...,...,...,...,...,...
29670,313341432,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,"Built on the MOEN M-PACT common valve system, ...",Colinet
29671,313341429,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,Brushed gold finish adds a touch of sophistica...,Colinet
29672,313341430,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,"Built on the MOEN M-pact common valve system, ...",Colinet
29673,313341434,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,Brushed nickel finish provides a lightly brush...,Colinet


In [71]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [72]:
train_contexts[1][207:226],len('singing and dancing')

('singing and dancing', 19)

In [168]:
train_answers[:10]

[{'text': 'in the late 1990s', 'answer_start': 269, 'answer_end': 286},
 {'text': 'singing and dancing', 'answer_start': 207, 'answer_end': 226},
 {'text': '2003', 'answer_start': 526, 'answer_end': 530},
 {'text': 'Houston, Texas', 'answer_start': 166, 'answer_end': 180},
 {'text': 'late 1990s', 'answer_start': 276, 'answer_end': 286},
 {'text': "Destiny's Child", 'answer_start': 320, 'answer_end': 335},
 {'text': 'Dangerously in Love', 'answer_start': 505, 'answer_end': 524},
 {'text': 'Mathew Knowles', 'answer_start': 360, 'answer_end': 374},
 {'text': 'late 1990s', 'answer_start': 276, 'answer_end': 286},
 {'text': 'lead singer', 'answer_start': 290, 'answer_end': 301}]

In [73]:
myData = myData.loc[:,['omsid','product_name','series_name']]

In [74]:
myData.loc[:,'answer_start'] = "N"
myData.loc[:,'answer_end'] = "N"

In [75]:
counter = 0
for i, row in myData.iterrows():
    if row.series_name in row.product_name:
        myData.loc[i,'answer_start'] = row.product_name.index(row.series_name)
        myData.loc[i,'answer_end'] = myData.loc[i,'answer_start'] + len(row.series_name)
        counter += 1
print(f"out of {myData.shape[0]}, {counter} are in the title")

out of 29675, 23391 are in the title


In [172]:
23391/29675

0.7882392586352148

In [76]:
myData_cleaned = myData.loc[myData.loc[:,'answer_start'] != "N", :]

In [77]:
myData_oos = myData.loc[myData.loc[:,'answer_start'] == "N", :]

In [12]:
myData_oos

,omsid,product_name,series_name,answer_start,answer_end
0,313296145,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
1,313296138,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
2,313296149,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
3,313296158,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
13,313026686,WALTZ 8 in. Widespread 2-Handle Lavatory Bathr...,Waltz,N,N
...,...,...,...,...,...
29660,305303412,1-Spray 1 in. Single Wall Mount Waterfall Hand...,Handheld Shower Heads,N,N
29663,314630664,Geneva 30 in. Farmhouse Apron Front Kitchen Si...,GENEVA,N,N
29664,314630667,Champlain 30 in. Farmhouse Apron Front Kitchen...,CHAMPLAIN,N,N
29665,314630665,Fortress 29 3/4 in. White Farmhouse Apron Fron...,ROUND APRON,N,N


In [78]:
from sklearn.model_selection import train_test_split

In [79]:
train, test = train_test_split(myData_cleaned, test_size=0.2)

In [80]:
train_answers, train_contexts, train_questions = [], [], []
for i, row in train.iterrows():
    train_contexts.append(row.product_name)
    train_answers.append({'text': row.series_name, 'answer_start': row.answer_start, 'answer_end': row.answer_end})
    train_questions.append('What is the series name of this product?')
    
test_answers, test_contexts, test_questions = [], [], []
for i, row in test.iterrows():
    test_contexts.append(row.product_name)
    test_answers.append({'text': row.series_name, 'answer_start': row.answer_start, 'answer_end': row.answer_end})
    test_questions.append('What is the series name of this product?')
    

In [81]:
## out of sample data set...those whose series names do not exist in product titles
test_oos_answers, test_oos_contexts, test_oos_questions = [], [], []
for i, row in myData_oos.iterrows():
    test_oos_contexts.append(row.product_name)
    test_oos_answers.append({'text': row.series_name, 'answer_start': None, 'answer_end': None})
    test_oos_questions.append('What is the series name of this product?')

In [76]:
ls roberta-base-squad2/

README.md           merges.txt         special_tokens_map.json
config.json         pytorch_model.bin  tokenizer_config.json
flax_model.msgpack  rust_model.ot      vocab.json


In [93]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

In [82]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, test_answers)

In [94]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
#train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [85]:
from transformers import DistilBertForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
# model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

In [19]:
# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 2: 100%|██████████| 1170/1170 [01:55<00:00, 10.14it/s, loss=0.00322] 


In [86]:
from transformers import pipeline

In [87]:
qa = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [77]:
question="What is the series name of this product?"

In [95]:
# switch model out of training mode
model.to(device)
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# initialize validation set data loader
val_loader = DataLoader(val_dataset, batch_size=16)
# initialize list to store accuracies
acc = []
# loop through batches
all_preds, all_truths = [],[]
for batch in val_loader:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # we will use true positions for accuracy calc
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull prediction tensors out and argmax to get predicted tokens
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
        pred_answer_batch = [ tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[x][start_pred[x]:end_pred[x]+1])) for x in range(input_ids.shape[0])]
        true_answer_batch = [ tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[x][start_true[x]:end_true[x]+1])) for x in range(input_ids.shape[0])]
        all_preds.extend(pred_answer_batch)
        all_truths.extend(truth_batch)
# calculate average accuracy in total
acc = sum(acc)/len(acc)

In [98]:
test.loc[:,'series_name_pred'] = all_preds

In [103]:
test['agreement'] = test['series_name_pred'] == test['series_name'].apply(lambda x: x.lower())

In [107]:
test.to_csv('validation_dataset_results.csv',index=False)

In [108]:
## break down accuracy by categories...

In [111]:
client=bigquery.Client(project='analytics-online-data-sci-thd')
query = """
with cat_names as (
select omsid, l1, l2, taxonomy  from `hd-personalization-prod.personalization.common_CATGPENREL_taxonomy` ),

series_name_qa_model_results as (
select *, case when agreement=true then 1 else 0 end as num_agree  from `analytics-online-data-sci-thd.tianlong.series_name_qa_model_results` )

select l1,l2,  avg(num_agree) categorywise_accuracy from series_name_qa_model_results a join cat_names b on safe_cast(a.omsid as string) = b.omsid 
group by l1, l2
order by categorywise_accuracy
"""
results = client.query(query).to_dataframe()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [112]:
results

,l1,l2,categorywise_accuracy
0,Featured Products,Holiday Exclusions 2020,0.700000
1,Appliances,Ranges,0.750000
2,Featured Products,Up to 20 Percent Off MOEN Essentials,0.769231
3,Appliances,Cooktops,0.800000
4,Appliances,Microwaves,0.833333
...,...,...,...
116,Featured Products,Pro Appreciation Glacier Bay Bath,1.000000
117,Featured Products,Delta Faucet New Products,1.000000
118,Featured Products,Email offer,1.000000
119,Featured Products,EndCap New Lower Prices,1.000000


In [114]:
## see some wrong cases
test.loc[test.loc[:,'agreement']==False]

,omsid,product_name,series_name,answer_start,answer_end,series_name_pred,agreement
11781,203095371,Iron/Tones Drop-In/Undermount Cast-Iron 17 in....,Iron/Tones,0,10,iron / tones,False
106,303404502,Enbrighten Z-Wave Plus In-Wall 1000W Smart Dimmer,Z-Wave Plus,11,22,z - wave plus,False
22975,313966201,Bel-Air 8 in. Widespread 2-Handle Bathroom Fau...,Bel-Air,0,7,bel - air,False
26594,306778317,Studio S Single-Handle Single Hole Vessel Bath...,Studio S,0,8,studio,False
7580,206665129,Pax Zero-Radius 22.5in. 16 Gauge Undermount Si...,Pax Zero-Radius,0,15,pax zero - radius,False
...,...,...,...,...,...,...,...
11777,203095380,Iron/Tones Drop-In/Undermount Cast-Iron 24 in....,Iron/Tones,0,10,iron / tones,False
17015,315709525,Richard White Ceramic 33.27 in. Double Bowl Fa...,Richard White Ceramic 33.27 in. Double Bowl Fa...,0,84,richard white ceramic 33. 27 in. double bowl f...,False
22972,313966233,Bel-Air 8 in. Widespread 2-Handle Bathroom Fau...,Bel-Air,0,7,bel - air,False
16159,313232069,Metris S Single Hole 1-Handle Mid-Arc Bathroom...,Metris,0,6,metris s,False


For out of sample data sets

In [ ]:
## evaluate out of sample testing set
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
val_encodings = tokenizer(test_oos_contexts, test_oos_questions, truncation=True, padding=True)

val_dataset = SquadDataset(val_encodings)

In [ ]:
# switch model out of training mode
model.to(device)
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# initialize validation set data loader
val_loader = DataLoader(val_dataset, batch_size=16)
# initialize list to store accuracies
acc = []
# loop through batches
all_preds, all_truths = [],[]
for batch in val_loader:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # we will use true positions for accuracy calc
        # start_true = batch['start_positions'].to(device)
        # end_true = batch['end_positions'].to(device)
        # truth_batch = batch['true_aws']
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull prediction tensors out and argmax to get predicted tokens
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        # acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        # acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
        pred_answer_batch = [ tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[x][start_pred[x]:end_pred[x]+1])) for x in range(input_ids.shape[0])]
        # true_answer_batch = [ tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[x][start_true[x]:end_true[x]+1])) for x in range(input_ids.shape[0])]
        all_preds.extend(pred_answer_batch)
        # all_truths.extend(truth_batch)
# calculate average accuracy in total
# acc = sum(acc)/len(acc)

In [92]:
counter = 0
for i, pred in enumerate(all_preds):
    if pred == myData_oos.series_name.values[i].lower():
        # print(pred,"++++", myData_oos.series_name.values[i].lower())
        counter+=1
print(counter/len(all_preds))

0.0549013367281986


In [60]:
np.sum(list(map(lambda x:x.lower(),myData_oos.series_name.values)) == np.array(all_preds))/len(all_preds)

0.0549013367281986

In [160]:
## how to save model
model_path = "distilbert-custom/"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('distilbert-custom/tokenizer_config.json',
 'distilbert-custom/special_tokens_map.json',
 'distilbert-custom/vocab.txt',
 'distilbert-custom/added_tokens.json',
 'distilbert-custom/tokenizer.json')

In [20]:
## how to load pretrained models...
model_path = 'distilbert-custom/'
model = DistilBertForQuestionAnswering.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)

In [197]:
test.loc[:,'naive'] = test['product_name'].apply(lambda x: x.split()[0].lower())
test.loc[:,'series_name_lower'] = test['series_name'].apply(lambda x: x.lower())

In [201]:
## naive approach, using first word of title as the series name...
## accuracy around 81%, which indicates the model did learn something non-trivial...
np.sum(np.array(test.series_name_lower.values) == np.array(test.naive.values))/len(all_preds)

0.8179098097884163